In [ ]:
import os
import json
import pandas as pd
import multiprocessing

from aco.aco import ACO

In [ ]:
def update_local_stats(solutions):
    cols  = ["best", "worst", "mean", "std", "mean_size"]
    stats = pd.DataFrame(columns = cols)

    for gen in range(1, max_iter + 1):

        best  = solutions[gen]["cost"].max()
        worst = solutions[gen]["cost"].min()
        mean  = solutions[gen]["cost"].mean()
        std   = solutions[gen]["cost"].std()

        mean_size  = solutions[gen]["path"].apply(len).mean()

        to_add = {"best": best, "worst": worst, "mean": mean, "std": std, "mean_size": mean_size}
        stats  = stats.append(to_add, ignore_index = True)
        
    return stats

In [ ]:
def run_aco(rep):
    solutions = colony.longest_path(first_node, last_node, pheromone_init, max_iter, 
                                        n_ants, alpha, beta, evaporation_rate, update) 
    return solutions

In [ ]:
folder_path = "dataset"

data         = pd.read_csv(os.path.join(folder_path, "entrada1.txt"), sep = "\t", header = None)
data.columns = ["n1", "n2", "w"]

data.head(3)

In [ ]:
#### Parâmetros testados
# 1. update           = "global", "local", "global-local", "all"
# 2. n_ants           = 10, 50, 100, 500
# 3. max_iter         = 10, 30, 50
# 4. pheromone_init   = 1, 0.5, 2
# 5. alpha            = 1, 0.5, 0.05
# 6. beta             = 1, 0.5, 0.05
# 7. evaporation_rate = 0.05, 0.5, 1

n_ants           = 500
max_iter         = 100
pheromone_init   = 0.5
alpha            = 1       # pheromone concentration
beta             = 1       # function quality when calculating prob. transition
evaporation_rate = 0.05    # pheromone

update           = "global-local"

colony           = ACO(data)

# you can fix the nodes, just put the number of the node
# ex: first_node = 1
first_node       = min(colony.vertices)   #"random"
last_node        = max(colony.vertices)   #"random"

n_evaluations    = 30

In [ ]:
with multiprocessing.Pool() as pool:
        solutions = pool.map(run_aco, range(1, n_evaluations + 1))

In [ ]:
#### to save the experiment values
# they will be saved in a folder called "results/"

test_number = 34

repetitions = {}

for rep in range(1, n_evaluations + 1):
    repetitions[rep] = update_local_stats(solutions[rep - 1])
    
filename      = "experiment00{}.json".format(test_number)
where_to_save = os.path.join("results", filename)

os.makedirs(os.path.dirname(where_to_save), exist_ok = True) 

data_dict     = {key: repetitions[key].to_dict(orient = 'index') for key in repetitions.keys()}

with open(where_to_save, 'w') as fp:
    json.dump(data_dict, fp, indent = 4, sort_keys = True)